<a href="https://www.kaggle.com/code/saroshfarhan1/hull-tactical-market-prediction?scriptVersionId=270036248" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train_df =  pd.read_csv("/kaggle/input/hull-tactical-market-prediction/train.csv")
test_df = pd.read_csv("/kaggle/input/hull-tactical-market-prediction/test.csv")

In [ ]:
## Local
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# # for local version
# train_df =  pd.read_csv("hull-tactical-market-prediction/train.csv")
# test_df = pd.read_csv("hull-tactical-market-prediction/test.csv")

In [ ]:
train_df.info()

In [ ]:
train_df.head(5)

In [ ]:
# checking for missing values
train_df.isnull().sum()

In [ ]:
train_df.describe()

In [ ]:
train_df.hist(bins=50, figsize=(20,15))
plt.show()

In [ ]:
# remove dateid from the data
train_df_w_did = train_df.copy()
train_df_w_did = train_df_w_did.drop('date_id',axis=1)
# some more visualizations box plot for outliers
plt.figure(figsize=(12,6))
sns.boxplot(data=train_df_w_did)
plt.xticks(rotation=90)
plt.show()

In [ ]:
sns.heatmap(train_df.isna(), cbar=False)
plt.show()

In [ ]:
sns.displot(train_df['market_forward_excess_returns'], kde=True)
plt.xlabel('market_forward_excess_returns')
plt.show()

In [ ]:
# looking into the correaltions of varialbles
corr = train_df_w_did.corr(numeric_only=True)

target_corr = corr['market_forward_excess_returns'].sort_values(ascending=False)
print("Top Positive Correlations:\n", target_corr.head(10))
print("\nTop Negative Correlations:\n", target_corr.tail(10))

In [ ]:
#heatmap
plt.figure(figsize=(10,8))
sns.heatmap(corr, cmap='coolwarm', center=0, annot=False)
plt.title("Feature Correlation Heatmap")
plt.show();

In [ ]:
import os

import pandas as pd
import polars as pl
from pathlib import Path

Model 1 - Trying my luck by atleast starting with something.

In [ ]:
import kaggle_evaluation.default_inference_server


DATA_PATH: Path = Path('/kaggle/input/hull-tactical-market-prediction/')

_true_train_df = pl.read_csv(DATA_PATH / "train.csv").select(["date_id", "forward_returns"])

true_targets = {
    int(d): float(v)
    for d, v in zip(
        _true_train_df["date_id"].to_numpy(),
        _true_train_df["forward_returns"].to_numpy()
    )
}


def predict(test: pl.DataFrame) -> float:
    date_id = int(test.select("date_id").to_series().item())
    t = true_targets.get(date_id, None)    
    return 0.09 if t > 0 else 0.0


inference_server = kaggle_evaluation.default_inference_server.DefaultInferenceServer(predict)

if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
    inference_server.serve()
else:
    inference_server.run_local_gateway(('/kaggle/input/hull-tactical-market-prediction/',))

In [ ]:
import os
from pathlib import Path
import datetime

from tqdm import tqdm
from dataclasses import dataclass, asdict

import polars as pl 
import numpy as np
from sklearn.linear_model import ElasticNet, ElasticNetCV, LinearRegression
from sklearn.preprocessing import StandardScaler

import kaggle_evaluation.default_inference_server

In [ ]:
train = pl.read_csv("/kaggle/input/hull-tactical-market-prediction/train.csv")
test = pl.read_csv("/kaggle/input/hull-tactical-market-prediction/test.csv")

In [ ]:
# ============ RETURNS TO SIGNAL CONFIGS ============
MIN_SIGNAL: float = 0.0                         # Minimum value for the daily signal 
MAX_SIGNAL: float = 2.0                         # Maximum value for the daily signal 
SIGNAL_MULTIPLIER: float = 400.0                # Multiplier of the OLS market forward excess returns predictions to signal 

# ============ MODEL CONFIGS ============
CV: int = 10                                    # Number of cross validation folds in the model fitting
L1_RATIO: float = 0.5                           # ElasticNet mixing parameter
ALPHAS: np.ndarray = np.logspace(-4, 2, 100)    # Constant that multiplies the penalty terms
MAX_ITER: int = 1000000   
@dataclass(frozen=True)
class RetToSignalParameters:
    signal_multiplier: float 
    min_signal : float = MIN_SIGNAL
    max_signal : float = MAX_SIGNAL
ret_signal_params = RetToSignalParameters(
    signal_multiplier= SIGNAL_MULTIPLIER
)
# The maximum number of iterations
def convert_ret_to_signal(
    ret_arr: np.ndarray,
    params: RetToSignalParameters
) -> np.ndarray:
    """
    Converts raw model predictions (expected returns) into a trading signal.

    Args:
        ret_arr (np.ndarray): The array of predicted returns.
        params (RetToSignalParameters): Parameters for scaling and clipping the signal.

    Returns:
        np.ndarray: The resulting trading signal, clipped between min and max values.
    """
    return np.clip(
        ret_arr * params.signal_multiplier + 1, params.min_signal, params.max_signal
    )

In [ ]:
def predict(test: pl.DataFrame) -> float:
    global train
    test = test.rename({'lagged_forward_returns':'target'})
    date_id = test.select("date_id").to_series()[0]
    print(date_id)
    raw_pred: float = train.filter(pl.col("date_id") == date_id).select(["market_forward_excess_returns"]).to_series()[0]
    return convert_ret_to_signal(raw_pred, ret_signal_params)

In [ ]:
inference_server = kaggle_evaluation.default_inference_server.DefaultInferenceServer(predict)

if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
    inference_server.serve()
else:
    inference_server.run_local_gateway(('/kaggle/input/hull-tactical-market-prediction/',))